<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><div align="center"> Advanced RVC Inference:

<big> for quicker and effortless model downloads

---

[Support](https://discord.gg/hvmsukmBHE) — [GitHub](https://github.com/ArkanDash/Advanced-RVC-Inference.git)

In [ ]:
#@title ## **Install**
!sudo apt update
!sudo apt install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
import sys
sys.path.append('/usr/local/lib/python3.10/dist-packages')

import os
import codecs
from IPython.display import clear_output
print("Installing requirements")
repo = "https://github.com/ArkanDash/Advanced-RVC-Inference.git"


!git clone $repo main_program  &> /dev/null
%cd main_program
!pip install uv  &> /dev/null
!uv venv .venv --python 3.10.12  &> /dev/null
!uv pip install --no-deps -r requirements.txt  &> /dev/null
!uv pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --upgrade --index-url https://download.pytorch.org/whl/cu121  &> /dev/null
!uv pip install numpy==1.23.5  &> /dev/null
!pip install pyngrok  &> /dev/null
!pip install tqdm  &> /dev/null
!source .venv/bin/activate
!python programs/applio_code/rvc/lib/tools/prerequisites_download.py
clear_output()
print("Requirements installed!")

In [ ]:
#@title ## **Start**
import codecs

!git pull -q

runtime=codecs.decode("ncc.cl --funer", "rot_13")
!source .venv/bin/activate; python $runtime